In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import gensim

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
data_root = '/home/thenhz/workspace/hate-speech/data/'

GLOVE_MODEL_FILE = data_root + 'word2vec/glove_WIKI'


word2vec_model = gensim.models.Word2Vec.load(GLOVE_MODEL_FILE)
print("word2vec model loaded...")

word2vec model loaded...


In [6]:
data = pd.read_csv('data/hate-speech/haspeede_TW-train.tsv', sep='\t', names=['text','sentiment'])
print("data file loaded..")

FLAGS = re.MULTILINE | re.DOTALL

#preprocess

def preprocess_tweet(text,FLAGS = FLAGS):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)
    
    def hashtag(text):
        text = text.group()
        hashtag_body = text[1:]
        if hashtag_body.isupper():
            result = u"<hashtag> {} <allcaps>".format(hashtag_body)
        else:
            result = " ".join(["<hashtag>"]) #+ re.split(u"(?=[A-Z])", hashtag_body, flags=FLAGS))
        return result

    def allcaps(text):
        text = text.group()
        return text.lower() + " <allcaps>"

    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<NHZ_URL>")
    text = re_sub(r"/"," / ")
    text = re_sub(r"@\w+", "<NHZ_USER>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<NHZ_SMILE>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<NHZ_LOL_FACE>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<NHZ_SAD_FACE>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<NHZ_NEUTRAL_FACE>")
    text = re_sub(r"<3","<NHZ_HEART>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<NHZ_NUMBERS>")
    text = re_sub(r"#\S+", "<NHZ_HASHTAG>")
    text = re_sub(r"([!?.]){2,}", "")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <NHZ_ELONG>")

    ## -- I just don't understand why the Ruby script adds <allcaps> to everything so I limited the selection.
    # text = re_sub(r"([^a-z0-9()<>'`\-]){2,}", allcaps)
    #text = re_sub(r"([A-Z]){2,}", allcaps)
    
    return text

def getStopWords(fileName='/home/thenhz/workspace/hate-speech/data/stopwords/IT-stopwordsISO-stopwords.txt'):
    with open(fileName) as myFile:
        lines = myFile.read().splitlines()
    return lines

def filter_stop_words(train_sentences, stop_words):
    count = 0
    for i, sentence in train_sentences.iterrows():
        #print(sentence['text'])
        #if(count>10):
        #    break
        #count+=1
        new_sent = preprocess_tweet(sentence['text'])
        #print(new_sent)
        new_sent = ' '.join([word for word in new_sent.split() if word not in stop_words])
        #print(new_sent)
        train_sentences.set_value(i, 'text2', new_sent, takeable=False)
    return train_sentences

stop_words = getStopWords()
train_sentences = filter_stop_words(data, stop_words)

data file loaded..


/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:65: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [7]:
train_sentences

,text,sentiment,text2
576268275560116224,@matteorenzi ...all'invasione di questi animal...,1,"NHZ_USER all'invasione animali, offesa animali..."
815833978691260416,"È terrorismo anche questo, per mettere in uno ...",0,"È terrorismo questo, mettere soggezione render..."
815112299044372480,@Nanoalto @FedeAngeli infatti finché ci hanno ...,0,NHZ_USER NHZ_USER finché guadagnato campi <has...
577451440047853568,@dinofarnesi Piovegovernolad In Italia a...,1,NHZ_USER Piovegovernolad <NHZ_ELONG>In Italia ...
844921803575382016,#londonattack chiedete ai buonisti del cavolo ...,1,<hashtag> chiedete buonisti cavolo vogliono is...
811099342593462272,"Corriere: Tangenti, Mafia Capitale dimenticata...",0,"Corriere: Tangenti, Mafia Capitale dimenticata..."
833323806873833473,"@Pontifex_it ad uno ad uno, perché quando i mi...",0,"NHZ_USER uno, migranti israeliti arrivarono te..."
835493194796314624,Il divertimento del giorno? Trovare i patrioti...,0,Il divertimento giorno? Trovare patrioti itali...
839605324684361730,Modena: Comune paga la benzina ai nomadi che p...,0,Modena: Comune paga benzina nomadi portano fig...
851906322283016192,@Cartabiancarai3 @KhalidChaouki3 altro che isl...,0,NHZ_USER NHZ_USER islam cristianesimo dobbiamo...


In [39]:
#data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 0].size) #positive
print(data[ data['sentiment'] == 1].size) #negative



4052
1944


In [40]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/site-packages/keras_preprocessing/text.py:175: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [41]:
len(tokenizer.word_index)

8558

In [42]:
word_vectors = word2vec_model.wv
MAX_NB_WORDS = len(word_vectors.vocab)
MAX_SEQUENCE_LENGTH = 200


In [43]:
len(word_vectors.vocab)

730613

In [44]:
WV_DIM = 100

# we initialize the matrix with random numbers
wv_matrix = (np.random.rand(max_fatures, WV_DIM) - 0.5) / 5.0
for word, i in tokenizer.word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        # words not found in embedding index will be all-zeros.
        wv_matrix[i] = embedding_vector
    except:
        pass 

In [45]:
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, WV_DIM,weights=[wv_matrix], input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

#model = Sequential()
#model.add(LSTM(32, return_sequences=True,
#               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
#model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
#model.add(LSTM(32))  # return a single vector of dimension 32
#model.add(Dense(10, activation='softmax'))

#model.compile(loss='categorical_crossentropy',
#              optimizer='rmsprop',
#              metrics=['accuracy'])

/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  after removing the cwd from sys.path.
/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  """


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 37, 100)           200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               232848    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 433,242
Trainable params: 433,242
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2008, 37) (2008, 2)
(990, 37) (990, 2)


In [ ]:
batch_size = 32
model.fit(X_train, Y_train, nb_epoch = 10, validation_split = 0.1, batch_size=batch_size, verbose = 2)

/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 1807 samples, validate on 201 samples
Epoch 1/10
 - 5s - loss: 0.5771 - acc: 0.6973 - val_loss: 0.5845 - val_acc: 0.6617
Epoch 2/10
 - 4s - loss: 0.4659 - acc: 0.7947 - val_loss: 0.5181 - val_acc: 0.7612
Epoch 3/10
 - 4s - loss: 0.3472 - acc: 0.8556 - val_loss: 0.5312 - val_acc: 0.7512
Epoch 4/10
 - 4s - loss: 0.2672 - acc: 0.9032 - val_loss: 0.5963 - val_acc: 0.7313
Epoch 5/10
 - 4s - loss: 0.2021 - acc: 0.9181 - val_loss: 0.6662 - val_acc: 0.7463
Epoch 6/10


In [20]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score = model.evaluate(X_test, Y_test, batch_size = batch_size)
#logger.info('mse=%f, mae=%f, mape=%f' % (scores[0],scores[1],scores[2]))
#print(score)
#print("score: %.2f " % (score))
#print("acc: %.2f" % (acc))
print(score)

[]


In [21]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 55.5921052631579 %
neg_acc 84.54810495626822 %
